In [ ]:
%pip install transformers
%pip install accelerate>=0.26.0
%pip install --upgrade jinja2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

os.environ["HF_HOME"] = "/tmp/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/tmp/huggingface"


model_name = "Qwen/Qwen2.5-7B-Instruct-1M"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    cache_dir="/tmp/huggingface"
)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="/tmp/huggingface")

prompt = '''You are given a passage from a movie script. Return all spoken dialogue. This may be in the form of explicit dialogue, or as a person speaking in stage directions or in any other sections of the script. Do not return any addition content aside from the explicit lines from the transcript. Do not number the output. 

MS -- Dorothy and Toto -- she tosses him a piece of the
cruller -- Toto eats it -- Dorothy speaks as she walks
forward -- she sings -- leans against haystack -- then walks
over near rake -- CAMERA PANS right --

                    DOROTHY (CONT'D)
          Do you suppose there is such a place,
          Toto? There must be. It's not a place
          you can get to by a boat or a train.
          It's far, far away -- behind the moon --
          beyond the rain --

                    DOROTHY (CONT'D)
              (sings)
          Somewhere, over the rainbow, way up high,
          There's a land that I heard of once in a
          lullaby.

Somewhere, over the rainbow, skies are blue, And the dreams
that you dare to dream really do.... CS -- Toto by wheel of
rake -- listening to song -- DOROTHY o.s. (sings) ...come
true.... MCS -- Dorothy singing -- swings on wheel of rake --
then walks forward around wheel -- Toto jumps up onto seat of
rake -- Dorothy pets him -- sits on front of rake -- CAMERA
PULLS back -- Dorothy finishes song --

                    DOROTHY (CONT'D)
              (sings)
          ...Someday I'll wish upon a star And wake
          up where the clouds are far behind me.

Where troubles melt like lemon drops, Away above the chimney
tops, That's where you'll find me. Somewhere, over the
rainbow, bluebirds fly. Birds fly over the rainbow, Why then -
- oh, why can't I? If happy little bluebirds fly Beyond the
rainbow Why, oh, why can't I? LS -- Miss Gulch rides along
country road on bicycle -- CAMERA PANS to right with her --
LS -- Miss Gulch rides forward to front of Gale's home --
stops and gets off her bicycle as Uncle Henry comes forward --

'''
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
